<a href="https://colab.research.google.com/github/CodyBurker/W266_Group_Project/blob/master/CNN_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on: https://machinelearningmastery.com/develop-word-embedding-model-predicting-movie-review-sentiment/

In [1]:
import pandas as pd
import string
from nltk.corpus import stopwords
import nltk
import numpy as np

In [2]:
!git clone https://github.com/CodyBurker/W266_Group_Project

Cloning into 'W266_Group_Project'...
remote: Enumerating objects: 186, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 186 (delta 77), reused 67 (delta 13), pack-reused 0
Receiving objects: 100% (186/186), 138.88 MiB | 10.52 MiB/s, done.
Resolving deltas: 100% (77/77), done.
Checking out files: 100% (41/41), done.


In [5]:
X_train = pd.read_csv("W266_Group_Project/x_train_stage_1_sampled_yelp_data.csv")
y_train = pd.read_csv("W266_Group_Project/y_train_stage_1_sampled_yelp_data.csv")
X_test = pd.read_csv("W266_Group_Project/x_test_sampled_yelp_data_NEW.csv")
y_test = pd.read_csv("W266_Group_Project/y_test_sampled_yelp_data_NEW.csv")
y_train = np.array(y_train.stars.astype('int')) - 1
y_test = np.array(y_test.stars.astype('int')) - 1

Encode text and pad it.

In [4]:
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['text'])


In [8]:
# Hyper parameters to tune later: maxlen, padding
sentence_len = 1024

In [11]:
from tensorflow import keras
model = keras.models.load_model('W266_Group_Project/CNNModel')
x_train_stage_2 = pd.read_csv("W266_Group_Project/x_test_sampled_yelp_data_NEW.csv")
x_train_stage_2_tok = tokenizer.texts_to_sequences(x_train_stage_2['text'])
x_train_stage_2_enc = pad_sequences(x_train_stage_2_tok, maxlen=sentence_len, padding='post', truncating = 'post')

# y_train_stage_2 = pd.read_csv("W266_Group_Project/y_train_stage_2_sampled_yelp_data.csv")
# y_train_stage_2_tok = tokenizer.texts_to_sequences(y_train_stage_2['text'])
# y_train_stage_2_enc = pad_sequences(x_train_stage_2_tok, maxlen=sentence_len, padding='post', truncating = 'post')


In [12]:
stage_2_preds = model.predict(x_train_stage_2_enc)

In [13]:
stage_2_preds[0:10]

array([[3.1926638e-07, 8.4973442e-07, 1.2803802e-04, 5.6307755e-02,
        9.4356310e-01],
       [3.9673426e-07, 6.1862315e-06, 1.8268750e-03, 2.7340049e-01,
        7.2476602e-01],
       [1.5019916e-07, 1.4409806e-06, 4.6640221e-04, 1.5705076e-01,
        8.4248126e-01],
       [5.1040797e-07, 1.1121765e-06, 1.5059375e-04, 5.6604505e-02,
        9.4324332e-01],
       [1.6187517e-04, 3.9040970e-04, 8.3310986e-03, 1.9678921e-01,
        7.9432738e-01],
       [6.0401903e-06, 3.0412317e-05, 2.7157126e-03, 2.2024938e-01,
        7.7699846e-01],
       [8.7334290e-03, 4.8385742e-03, 2.1619990e-02, 1.5556428e-01,
        8.0924380e-01],
       [1.6305953e-03, 6.7020371e-03, 9.1605790e-02, 5.1989555e-01,
        3.8016605e-01],
       [1.5201421e-05, 7.9249339e-06, 1.8598697e-04, 2.8272552e-02,
        9.7151834e-01],
       [5.4163377e-07, 3.6022911e-06, 7.7499083e-04, 1.5937975e-01,
        8.3984107e-01]], dtype=float32)

In [14]:
preds_processed = np.argmax(stage_2_preds,axis=1) +1

In [15]:
preds_processed[0:10]

array([5, 5, 5, 5, 5, 5, 5, 4, 5, 5])

In [17]:
# Write processed preds to disk
pd.DataFrame({'CNN_predictions':preds_processed}).to_csv("W266_Group_Project/X_test_predictions_1_to_5_CNN.csv")